In [ ]:
import os
import requests
import math
import json

In [ ]:
API_KEY = "API_KEY"
JSON_PATH = "/Users/javonnii/dolby_io/test.json"
DATASET_PATH = "path to dataset"
url = "https://api.dolby.com/media/input" # to upload to Dolby's S3 container

# Upload audio file to Dolby I.O

In [ ]:
# check file path
for files in enumerate(os.path(DATASET_PATH)):
    print(files)

## accumulate dolby input locations for each class

In [ ]:
# accumulators
bowhead_whale = []
short_finned_pilot_whale = []
risso_dolphin = []
white_sided_dolphin = []
northern_right_whale = []
false_killer_whale = []
killer_whale = []
humpback_whale = []
sperm_whale = []
bowhead_whale = []

In [ ]:
# iterate through directories and upload audio files to S3

os.environ["PATH"] += os.pathsep + '/usr/local/bin'


def batch_operations(loc):
    for f in os.listdir(DATASET_PATH):
        if f not in (".ipynb_checkpoints"):
            file_path = os.path.join(DATASET_PATH, f)
            headers = {
                "x-api-key": API_KEY,
                "Content-Type": "application/json",
                "Accept": "application/json",
            }

            f = f.replace(" ", "_")

            body = {
                "url": f"dlb://in/{f}",
            }

            X = body.get("url")
            loc.append(X)

            response = requests.post(url, json=body, headers=headers)
            response.raise_for_status()
            data = response.json()
            presigned_url = data["url"]

            print("Uploading {0} to {1}".format(file_path, presigned_url))
            with open(file_path, "rb") as input_file:
                requests.put(presigned_url, data=input_file)

In [ ]:
batch_operations(white_sided_dolphin)

# Start Here for Processing

# Make an Enhance Request

In [ ]:
jobs = {}

for file in sperm_whale: # change animal name here

    jobs[file[9:]] ={
        'wav': file,
        'job_id': None,
        'status': "Pending",
        'response': None,
    }

In [ ]:
# double checking the dolby locations to ensure proper syntax
for k, v in jobs.items():
    print(v['wav'])

In [ ]:
# Good
def start_batch_processing(jobs):
    for k, v in jobs.items():
        print("Start Enhancing: {}".format(k))
        job_id = post_media_enhance(v['wav'])
        print(job_id)
        jobs[k]['job_id'] = job_id
    
    return jobs

In [ ]:
# Good
# create output accumulator to recall dolby output location for downloads
output_path = []

def post_media_enhance(loc): 
#     print(loc)
    body = {
        "input" : f"{loc}",
        "output": f"dlb://out/{loc[9:]}.enhance.json"
      }
    output_path.append(body['output'])

    url = "https://api.dolby.com/media/enhance"
    headers = {
      "x-api-key": API_KEY,
      "Content-Type": "application/json",
      "Accept": "application/json"
    }
    response = requests.post(url, json=body, headers=headers)
    response.raise_for_status()
    return response.json()["job_id"]

In [ ]:
X = start_batch_processing(jobs)

In [ ]:
#X
# make sure the output files look right! "dlb://out/Humpback_Whale.wav"
output_path

# Track Progress

In [ ]:
def get_url():
    return 'https://api.dolby.com/media/enhance'

In [ ]:
#Good

def get_media_enhance(job_id):
    url = get_url()
    headers = {
      "x-api-key": API_KEY,
      "Content-Type": "application/json",
      "Accept": "application/json"
    }
    params = {"job_id": job_id}

    response = requests.get(url, params=params, headers=headers)
    response.raise_for_status()
    data = response.json()
    return data

In [ ]:
#Good
import logging
import time

def check_job_status(jobs):
    # Check status until all jobs are complete
    active_jobs = True
    while active_jobs:
        active_jobs = False
        for name in jobs.keys():
            # Pending and Running status indicate the job is still processing
            status = jobs[name]['status']
            if status in {"Pending", "Running"}:
                # Check again to see if there has been a change since
                # the last loop.
                response = get_media_enhance(jobs[name]['job_id'])
                jobs[name]['status'] = response['status']
                if response['status'] in {"Pending", "Running"}:
                    active_jobs = True
                else:
                    # The job is complete, also store the response for later
                    logging.debug(response['status'])
                    jobs[name]['response'] = response

        # Wait a bit and then retry
        time.sleep(5)

    return jobs

In [ ]:
results = check_job_status(test)

In [ ]:
results

# Review the Output

In [ ]:
# Change the directory and filename here!!!

import shutil

# OUTPUT_PATH = "/Users/javonnii/dolby_io/enhanced/.wav"

url = "https://api.dolby.com/media/output"

headers = {
  "x-api-key": API_KEY,
  "Content-Type": "application/json",
  "Accept": "application/json"
}

for idx, out in enumerate(output_path):
    OUTPUT_PATH = f"/Users/javonnii/dolby_io/enhanced/sperm_whale/sperm_whale{idx}.wav"
    args = {
    "url": f"{out}"}
    
    with requests.get(url, params=args, headers=headers, stream=True) as response:
        response.raise_for_status()
        response.raw.decode_content = True
        print(type(response.raw))
        print("Downloading from {0} into {1}".format(response.url, OUTPUT_PATH))
        with open(OUTPUT_PATH, "wb") as output_file:
            shutil.copyfileobj(response.raw, output_file)